### This is to output the different types of deidentifier and explore the token choice

In [1]:
import os
import unicodedata
import re
from string import punctuation
import spacy
from spacy.en import English
import io
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999

In [2]:
df=pd.read_csv('/Users/leilei/Documents/DS1011Project/NOTEEVENTS.csv')

/Applications/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
nlp = spacy.load('en')
parser = English()

In [8]:
parsed=parser(df.iloc[1,-1].replace('**','__'))
for i in parsed[:15]:
    print(i.orth_)
#from this result we decide to padding these deidentifier first with ALL UPPERCASE and then tokenization

Admission
Date
:
 
[
_
_
2118
-
6
-
2
_
_
]


In [ ]:
##This function is findding the types of deidentifier in the dataset and see how many kinds of padding do we actually need
def identify(x):
    '''
    x: a list of tokens which is split by space in notes
    return: tokens which is in the format [**X**]
    '''
    temp_x = reconcat_list(x)
    return [i for i in temp_x if i.startswith('[')]
    
def reconcat_list(li):
    '''
    li: the list of tokens which is splitted by space in space
    return: token list which has been processed. the [**X**] should be joint as one token
    '''
    #print(li)
    new_li = []
    flag = False
    for i in li:
        #print(i)
        if i[0] != "[" and flag == False:
            new_li.append(i)
        elif i[0] == '[' and i[-1] == ']':
            new_li.append(i)
        elif i[0] == '[':
            temp_li=[]
            temp_li.append(i)
            flag = True
        elif i[-1] == ']':
            temp_li.append(i)
            new_li.append(' '.join(temp_li))
            flag = False
        else:#flag == False and in between
            temp_li.append(i)
    return new_li

In [ ]:
target_li=[]

start=time.time()
for i in range(15000):
    temp_li = df.iloc[i,-1].split()
    all_individual = identify(temp_li)
    target_li.extend(all_individual)
end=time.time()

target_li.sort()

with open('./unidentifier.txt','w') as f:
    for i in target_li:
        f.write(i+'\n')

In [ ]:
#further cleaning for finding the types
target=[]
with open('./unidentifier.txt','r') as f:
    for line in f:
        target.append(line)

target=list(set(target))
target.sort()

with open('./new_unidentifier.txt','w') as f:
    for i in target:
        f.write(i+'\n')